In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import yaml
from logging import getLogger

from recbole.quick_start import run_recbole
# from recbole.quick_start import objective_function
from recbole.config import Config
from recbole.data import data_preparation, create_dataset
from recbole.trainer import HyperTuning
from recbole.utils import (
    get_model,
    get_trainer,
    # init_logger,
)
# from loguru import logger

import utils
from real_temporal import TimeCutoffDataset

# 1. Declarations & Definitions

## 1.1. Define flags and global variables

In [3]:
use_TimeCutoff = True
reproducible = True
seed = 42

## 1.2. Define configurations

Configuration for data, model, training and evaluation

In [4]:
config_dict = {
    # For model
    'model': "NPE",
    'embedding_size': 64,
    'dropout_prob': 0.3,
    'loss_type': "CE",

    # For data
    'dataset': 'ml-100k',
    'load_col': {"inter": ['user_id', 'item_id', 'timestamp']},
    'use_TimeCutoff': use_TimeCutoff,

    # For training
    'epochs': 500,
    'train_batch_size': 4096,
    'learning_rate': 1e-3,
    'train_neg_sample_args': None,
    
    # For evaluation
    'eval_batch_size': 4096,
    'metrics': ["Recall", "MRR", "NDCG", "Hit", "Precision"],
    'topk': 10,
    'valid_metric': 'MRR@10',

    # Environment
    'checkpoint_dir': utils.get_path_dir_ckpt(),
    'device': 'mps',
    'show_progress': True,
    'reproducibility': reproducible,
    'seed': seed,
}

if use_TimeCutoff is True:
    config_dict = {
        **config_dict,
        'eval_args': {
            "order": "TO",
            "split": {"CO": '886349689'},
            "group_by": 'user_id'
        },
    }
else:
    config_dict = {
        **config_dict,
        'eval_args': {
            "order": "TO",
            "split": { "LS": "valid_and_test" },
            "group_by": None
        },
    }

# 2. Train

## 2.1. Declare necessary components for training

In [5]:
config = Config(config_dict=config_dict)

# Define data related things
if use_TimeCutoff:
    dataset = TimeCutoffDataset(config)
else:
    dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

# Define model
model_name = config['model']
model = get_model(model_name)(config, train_data._dataset).to(config['device'])

# Define trainer
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

In [6]:
# init_logger(config)
logger = utils.get_logger()
# logger.add(utils.get_path_log())

print(" === Config === ")
logger.info(config)

print(" === Dataset === ")
logger.info(dataset)

print(" === Model === ")
logger.info(model)

2024-06-18 23:13:33.590 | INFO     | __main__:<module>:6 - 
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = /Users/macos/miniforge3/envs/py/lib/python3.10/site-packages/recbole/config/../dataset_example/ml-100k
checkpoint_dir = logs/Jun18_231331/ckpts
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 500
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'CO': '886349689'}, 'order': 'TO', 'group_by': 'user_id', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precis

 === Config === 
 === Dataset === 
 === Model === 


## 2.2. Start training

In [7]:
best_valid_score, best_valid_result = trainer.fit(
    train_data, 
    valid_data,
    verbose=True,
    show_progress=config["show_progress"]
)

print("** Validation result")
print(f"best_valid_score: {best_valid_score:.4f}")
for metric, val in best_valid_result.items():
    print(f"{metric:<15}: {val:.4f}")

Evaluate   : 100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 89.81it/s]

** Validation result
best_valid_score: 0.0183
recall@10      : 0.0603
mrr@10         : 0.0183
ndcg@10        : 0.0279
hit@10         : 0.0603
precision@10   : 0.0060


## 3.3. Start testing

In [8]:
test_result = trainer.evaluate(test_data)

print("** Test result")
for metric, val in test_result.items():
    print(f"{metric:<15}: {val:.4f}")

** Test result
recall@10      : 0.0079
mrr@10         : 0.0010
ndcg@10        : 0.0025
hit@10         : 0.0079
precision@10   : 0.0008


# 4. Tune hyper params

## 4.1. Define hyper params and object function

In [9]:
hyper_params = {
    'loguniform': {
        'learning_rate': [-8, 0]
    },
    'choice': {
        'embedding_size': [64],
        'dropout_prob': [0.2, 0.3, .5]
    }
}

In [10]:
def objective_function(config_dict=None, config_file_list=None):
    config = Config(
        config_dict=config_dict,
        config_file_list=config_file_list,
    )

    # Define data related things
    if use_TimeCutoff:
        dataset = TimeCutoffDataset(config)
    else:
        dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Define model
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])

    # Define trainer
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

    # Start training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)

    # Start evaluating
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

## 4.2. Save config as yaml file

In [11]:
with open(utils.get_path_conf(), 'w+') as f:
    yaml.dump(config_dict, f, allow_unicode=True)

## 4.2. Start tuning

In [13]:
tuning_algo = "bayes"
early_stop = 10
max_evals = 10

hp = HyperTuning(
    objective_function=objective_function,
    algo=tuning_algo,
    early_stop=early_stop,
    max_evals=max_evals,
    params_dict=hyper_params,
    fixed_config_file_list=[utils.get_path_conf()]
)


hp.run()

running parameters:                                   
{'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.0641808729888726}
current best valid score: 0.0171                      
current best valid result:                            
OrderedDict([('recall@10', 0.0541), ('mrr@10', 0.0171), ('ndcg@10', 0.0255), ('hit@10', 0.0541), ('precision@10', 0.0054)])
current test result:                                  
OrderedDict([('recall@10', 0.0), ('mrr@10', 0.0), ('ndcg@10', 0.0), ('hit@10', 0.0), ('precision@10', 0.0)])
running parameters:                                                  
{'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.01153580169569852}
running parameters:                                                  
{'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.001328144472054237}
running parameters:                                                  
{'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.05453059693665552}
current b

## 4.3. Export tunning result

In [14]:
# print best parameters
print('best params: ', hp.best_params)

# print best result
print('best result: ')
print(hp.params2result[hp.params2str(hp.best_params)])

# export to JSON file
tune_result = {
    'best_params': hp.best_params,
    'best_result': hp.params2result[hp.params2str(hp.best_params)]
}
with open(utils.get_path_tune_log(), "w+") as f:
    json.dump(tune_result, f, indent=2, ensure_ascii=False)

best params:  {'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.05453059693665552}
best result: 
{'model': 'NPE', 'best_valid_score': 0.0229, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.0526), ('mrr@10', 0.0229), ('ndcg@10', 0.0297), ('hit@10', 0.0526), ('precision@10', 0.0053)]), 'test_result': OrderedDict([('recall@10', 0.0157), ('mrr@10', 0.0118), ('ndcg@10', 0.0128), ('hit@10', 0.0157), ('precision@10', 0.0016)])}


# Dump

In [1]:
from real_temporal import run_recbole_with_TimeCutoff, TimeCutoffDataset

from recbole.trainer import HyperTuning
# from recbole.quick_start import objective_function
from recbole.config import Config
# from recbole.quick_start import run_recbole
from recbole.data import (
    create_dataset,
     data_preparation,
)

from recbole.utils import (
    get_model,
    get_trainer,
    init_seed,
)

def objective_function(config_dict=None, config_file_list=None):
    model_name = 'NPE'
    dataset_name = 'ml-100k'

    config = Config(
        config_dict=config_dict,
        config_file_list=config_file_list,
        model=model_name,
        dataset=dataset_name
    )
    # init_seed(config['seed'])
    dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

hp = HyperTuning(
    objective_function=objective_function,
    algo='bayes',
    early_stop=10,
    max_evals=100,
    params_file='model.hyper',
    fixed_config_file_list=['test.yml']
)

# run
hp.run()
# export result to the file
# hp.export_result(output_file='hyper_example.result')
# # print best parameters
# print('best params: ', hp.best_params)
# # print best result
# print('best result: ')
# print(hp.params2result[hp.params2str(hp.best_params)])

running parameters:                                    
{'embedding_size': 128, 'learning_rate': 0.012213481366920161, 'mlp_hidden_size': '[128,128]'}
current best valid score: 0.0164                       
current best valid result:                             
OrderedDict([('recall@10', 0.0668), ('mrr@10', 0.0164), ('ndcg@10', 0.0279), ('hit@10', 0.0668), ('precision@10', 0.0067)])
current test result:                                   
OrderedDict([('recall@10', 0.0435), ('mrr@10', 0.008), ('ndcg@10', 0.0161), ('hit@10', 0.0435), ('precision@10', 0.0043)])
running parameters:                                                     
{'embedding_size': 64, 'learning_rate': 0.09821758837521936, 'mlp_hidden_size': '[128,128]'}
current best valid score: 0.0218                                        
current best valid result:                                              
OrderedDict([('recall@10', 0.0647), ('mrr@10', 0.0218), ('ndcg@10', 0.0317), ('hit@10', 0.0647), ('precision@10', 0.0065)]

KeyboardInterrupt: 

In [5]:
from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

hp = HyperTuning(objective_function=objective_function, algo='bayes', early_stop=10,
                max_evals=100, params_file='model.hyper', fixed_config_file_list=['test.yml'])

# run
hp.run()
# export result to the file
hp.export_result(output_file='hyper_example.result')
# print best parameters
print('best params: ', hp.best_params)
# print best result
print('best result: ')
print(hp.params2result[hp.params2str(hp.best_params)])

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

17 Jun 22:12    INFO  build_posterior_wrapper took 0.000621 seconds

17 Jun 22:12    INFO  TPE using 0 trials



running parameters:                                    
{'embedding_size': 64, 'learning_rate': 0.0005927902936991883, 'mlp_hidden_size': '[128,128]'}
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

17 Jun 22:12    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

17 Jun 22:12    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'None', 'mode': {'valid': 'full', 'test': 'full'}}]

17 Jun 22:12    INFO  Loading model structure and parameters from saved/ItemKNN-Jun-17-2024_22-12-52.pth



current best valid score: 0.0463                       
current best valid result:                             
OrderedDict([('recall@10', 0.1262), ('mrr@10', 0.0463), ('ndcg@10', 0.0647), ('hit@10', 0.1262), ('precision@10', 0.0126)])
current test result:                                   
OrderedDict([('recall@10', 0.1145), ('mrr@10', 0.0473), ('ndcg@10', 0.0629), ('hit@10', 0.1145), ('precision@10', 0.0115)])
  1%|          | 1/100 [00:01<02:37,  1.59s/trial, best loss: -0.0463]

...

  9%|▉         | 9/100 [00:13<02:23,  1.58s/trial, best loss: -0.0463]

17 Jun 22:13    INFO  build_posterior_wrapper took 0.000442 seconds

17 Jun 22:13    INFO  TPE using 9/9 trials with best loss -0.046300



running parameters:                                                   
{'embedding_size': 64, 'learning_rate': 0.05115291209207257, 'mlp_hidden_size': '[128,128]'}
  9%|▉         | 9/100 [00:13<02:23,  1.58s/trial, best loss: -0.0463]

17 Jun 22:13    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

17 Jun 22:13    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'None', 'mode': {'valid': 'full', 'test': 'full'}}]

17 Jun 22:13    INFO  Loading model structure and parameters from saved/ItemKNN-Jun-17-2024_22-13-06.pth

17 Jun 22:13    INFO  Early stop triggered. Stopping iterations as condition is reach.



 10%|█         | 10/100 [00:15<02:18,  1.54s/trial, best loss: -0.0463]
best params:  {'embedding_size': 64, 'learning_rate': 0.0005927902936991883, 'mlp_hidden_size': '[128,128]'}
best result: 
{'model': 'ItemKNN', 'best_valid_score': 0.0463, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.1262), ('mrr@10', 0.0463), ('ndcg@10', 0.0647), ('hit@10', 0.1262), ('precision@10', 0.0126)]), 'test_result': OrderedDict([('recall@10', 0.1145), ('mrr@10', 0.0473), ('ndcg@10', 0.0629), ('hit@10', 0.1145), ('precision@10', 0.0115)])}


# 5. Offline evaluation

In [ ]:
EVAL_MODES = {
    "temporal": "TO",
    "leave1out": "LS"
}

In [ ]:
selected_eval_models = [EVAL_MODES[k] for k in ["temporal", "leave1out"]]
selected_eval_metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
selected_topk = 10

eval_args = {
    "order": "TO",
    "split": {
        'LS': "valid_and_test"
    },
    "group_by": None
}

config_dict = {
    'eval_args': eval_args,
    # 'train_neg_sample_args': None,
}

run_recbole(model=model, dataset=data, config_dict=config_dict)

In [ ]:
run_recbole(model='NPE', dataset='ml-100k', config_file_list=['test.yml'])